In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
import keras
import matplotlib.pyplot as plt  
import tqdm
import tensorflow as tf
import numpy as np

In [ ]:
model_path = 'final_model4.keras'
@keras.saving.register_keras_serializable()
class SumMinutesLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(SumMinutesLayer, self).__init__(**kwargs)
    
    def call(self, inputs):
        hours, minutes = inputs
        hours = tf.floor(hours)
        sum_min = hours * 60 + minutes
        return tf.stop_gradient(sum_min)
    
    def compute_output_shape(self, input_shape):
        return input_shape[0]
model = load_model(model_path, custom_objects={'SumMinutesLayer': SumMinutesLayer}, compile=False)

In [ ]:
folder_path = [
    'c:\\Users\\mymri\\repos\\clock_set/clock_01',      'c:\\Users\\mymri\\repos\\clock_set/clock_02_1',    'c:\\Users\\mymri\\repos\\clock_set/clock_02_2',
    'c:\\Users\\mymri\\repos\\clock_set/clock_03',      'c:\\Users\\mymri\\repos\\clock_set/clock_04',      'c:\\Users\\mymri\\repos\\clock_set/clock_05',
    'c:\\Users\\mymri\\repos\\clock_set/clock_06',      'c:\\Users\\mymri\\repos\\clock_set/clock_07',      'c:\\Users\\mymri\\repos\\clock_set/clock_08_1',
    'c:\\Users\\mymri\\repos\\clock_set/clock_08_3',    'c:\\Users\\mymri\\repos\\clock_set/clock_09_1',    'c:\\Users\\mymri\\repos\\clock_set/clock_09_2',
    'c:\\Users\\mymri\\repos\\clock_set/clock_09_2_1',  'c:\\Users\\mymri\\repos\\clock_set/clock_09_3_1',  'c:\\Users\\mymri\\repos\\clock_set/clock_09_3_2',
    'c:\\Users\\mymri\\repos\\clock_set/clock_09_3_3',  'c:\\Users\\mymri\\repos\\clock_set/clock_10',      'c:\\Users\\mymri\\repos\\clock_set/clock_11',
    'c:\\Users\\mymri\\repos\\clock_set/clock_12',      'c:\\Users\\mymri\\repos\\clock_set/clock_13',      'c:\\Users\\mymri\\repos\\clock_set/clock_14',
    'c:\\Users\\mymri\\repos\\clock_set/clock_15',      'c:\\Users\\mymri\\repos\\clock_set/clock_16',      'c:\\Users\\mymri\\repos\\clock_set/clock_17',
    'c:\\Users\\mymri\\repos\\clock_set/clock_18',      'c:\\Users\\mymri\\repos\\clock_set/clock_19',      'c:\\Users\\mymri\\repos\\clock_set/clock_20',
    'c:\\Users\\mymri\\repos\\clock_set/clock_21',      'c:\\Users\\mymri\\repos\\clock_set/clock_22',      'c:\\Users\\mymri\\repos\\clock_set/clock_23']

In [ ]:
def calculate_diff(y_true, y_pred):
    y_true = np.fmod(y_true, 720)
    y_pred = np.fmod(y_pred, 720)
    return (y_pred - y_true + 360) % 720 - 360
    

In [ ]:

total = 0
n = 0
diff_per_folder = []

for name in tqdm.tqdm(folder_path):
    labels_path = os.path.join(name, 'dataset_info.csv')
    labels_df = pd.read_csv(labels_path, usecols=[0, 1], header=None, names=['filename', 'label'])
    difference = []

    for index, row in tqdm.tqdm(labels_df.iterrows()):
        if index != 0:
            img_file = row['filename']
            true_label = row['label']
            true_label = float(true_label)

            img_path = os.path.join(name, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Ошибка при загрузке изображения: {img_path}")
                continue

            img = cv2.resize(img, (224, 224))  
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            img_array = img.astype('float32') / 255.0  
            img_array = np.expand_dims(img_array, axis=0)  

            predictions = model.predict(img_array,  verbose=0)

            diff = calculate_diff(true_label, predictions[2][0][0])
            difference.append(diff)
            accur = tf.abs(diff) <= 20
            total += int(accur)
            n += 1
    diff_per_folder.append(difference)
    print(f"acc after {name} = {total/n} ")

In [ ]:
n = len(diff_per_folder)
for i in range(0, 25):
    x = np.abs(diff_per_folder[i])
    count1 = np.sum(np.abs(x) < 5)
    count2 = np.sum(np.abs(x) < 20)
    print(f"{sum(x)/len(diff_per_folder[i])};{count1/len(x)};{count2/len(x)}")

In [ ]:
total = 0
n = 0

for i in range(len(diff_per_folder)):
    total += sum(np.abs(diff_per_folder[i]))
    n += len(diff_per_folder[i])

print(f"average diff = {total/n}") 
c5 = 0
c20 = 0
for i in range(len(diff_per_folder)):
    c5 += np.sum(np.abs(diff_per_folder[i]) < 5)
    c20 += np.sum(np.abs(diff_per_folder[i]) < 20)
print(c5/n)
print(c20/n)

In [ ]:
total = 0
n = 0
count_5 = 0
count_20 = 0
indexes = [1, 2, 6, 7, 19, 20, 24, 25, 26, 27, 28, 29]
for i in indexes:
    total += sum(np.abs(diff_per_folder[i]))
    n += len(diff_per_folder[i])
    count_5 += np.sum(np.abs(diff_per_folder[i]) < 5)
    count_20 += np.sum(np.abs(diff_per_folder[i]) < 20)

print(f"average diff = {total/n}") 
print(count_5/n)
print(count_20/n)